In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random
import copy

# dataset = np.random.uniform(-1,1, size=20).reshape(10,2)
# dataset

def target_function(): 
    target = np.random.uniform(-1,1, size=4).reshape(2,2)
    return target

# line = target_function()
# line


def find_boundary(p_x,p_y, line):
    det = (line[0][1] - line[0][0]) * (p_y - line[1][0]) - (line[1][1]-line[1][0]) * (p_x-line[0][0]) 
    return 1 if det > 0 else -1

def find_missclassified(weight,inserted, sign_list):
    missclassified = []
    for i in range(len(inserted)):
        product = np.matmul(weight, inserted[i]) 
        g_sign = 0 
        if product<0:g_sign = -1
        elif product>0:g_sign= 1
        if sign_list[i] !=g_sign:
            missclassified.append(i)
    return missclassified
    
def perceptron_la(weight, inserted, line, sign_list):
#     missclassified = find_missclassified(weight, inserted, line)
    counter = 1
    while True:
        missclassified = find_missclassified(weight,inserted, sign_list)
        if missclassified == []:
            break
        rand_choice = random.choice(inserted[missclassified])
        y_sign = find_boundary(rand_choice[1],rand_choice[2],line)
        weight += y_sign * rand_choice
        counter +=1
    return counter


def run_with_iter( num_of_iters, num_of_data):
    list_of_counters = []
    for i in range(num_of_iters):
        dataset = np.random.uniform(-1,1, size=num_of_data*2).reshape(num_of_data,2)
        line = target_function()
        signs_of_dataset = []
        for i in range(len(dataset)):
            signs_of_dataset.append(find_boundary(dataset[i][0],dataset[i][1],line))
        inserted = np.insert(dataset,0,1,axis=1)
        weight = np.zeros(3)
        count= perceptron_la(weight,inserted, line, signs_of_dataset) 
        list_of_counters.append(count)
    return list_of_counters



In [3]:
%%time
result = (run_with_iter(1000, 10))
np.average(result)

Wall time: 484 ms


12.496

In [4]:
%%time
result = (run_with_iter(1000, 100))
np.average(result)

Wall time: 21.4 s


99.905

In [104]:

def linear_regression(inserted, sign_list):
    x_pseudo = np.linalg.pinv(inserted)
    weight = np.matmul(x_pseudo, sign_list)
    predict = np.sign(np.matmul(weight, inserted.T)) 
    miss_count = 0
    for i in range(len(predict)):
        if int(predict[i]) != sign_list[i]:miss_count+=1
    #     print(predict, )
    #     print(sign_list, )
    return miss_count*1.0 / len(predict) ,weight

def run_linreg_with_iter(num_of_iters, data_size):
    list_of_probs = []
    for i in range(num_of_iters):
        dataset = np.random.uniform(-1,1, size=data_size*2).reshape(data_size,2)
        line = target_function()
        signs_of_dataset = []
        for i in range(len(dataset)):
            signs_of_dataset.append(find_boundary(dataset[i][0],dataset[i][1],line))
        inserted = np.insert(dataset,0,1,axis=1)
        weight = np.zeros(3)
        count, weight = linear_regression(inserted, signs_of_dataset) 
        list_of_probs.append(count)
    return list_of_probs, line, weight


# 8. c  Close to 0.01
result,line,weight = (run_linreg_with_iter(1000, 100))
np.average(result)

0.04045

In [105]:
def q9(line, weight, num_of_iter, num_of_fresh):
    fresh_probs = []
    for i in range( num_of_iter ):
        fresh_points = np.insert(np.random.uniform(-1,1, size=num_of_fresh*2).reshape(num_of_fresh,2),0,1,axis=1)

        signs_of_fresh = []
        for i in range(len(fresh_points)):
            signs_of_fresh.append(find_boundary(fresh_points[i][1],fresh_points[i][2],line))
        predict = np.sign(np.matmul(weight, fresh_points.T))

        miss_count = 0
        for i in range(len(predict)):
            if int(predict[i]) != int(signs_of_fresh[i]):miss_count+=1
        fresh_probs.append(miss_count / len(fresh_points))


    return np.average(fresh_probs)
# close to 0.01 answer C
q9(line,weight, 1000,1000)

0.137216

---

# Non Linear Transformation

## Question 11 


In [169]:
%%time
def nonlinear_target(x1, x2):
    return  1 if (x1**2 + x2**2 - 0.6) > 0 else -1

# training_set =  np.random.uniform(-1,1, size=1000*2).reshape(1000, 2)
# flipped_sign= []
# for i in range(len(training_set)):
#     sign = nonlinear_target(training_set[i][0],training_set[i][1])
#     if i %10 == 0:sign=-1*sign
#     flipped_sign.append(sign)

def q11():
    list_of_probs = []
    for i in range(1000):
        training_set =  np.random.uniform(-1,1, size=1000*2).reshape(1000, 2)
        flipped_sign= []
        for i in range(len(training_set)):
            sign = nonlinear_target(training_set[i][0],training_set[i][1])
            if i %10 == 0:sign=-1*sign
            flipped_sign.append(sign)
        inserted = np.insert(training_set,0,1,axis=1)
        count, weight = linear_regression(inserted, flipped_sign) 
        list_of_probs.append(count)
    return np.average(list_of_probs)
    
q11()


Wall time: 2.58 s


0.504888

## Question 12

In [223]:
np.set_printoptions(suppress=True)

In [225]:
def q12():
    weight_total= np.zeros(6)
    for i in range(1000):
        dataset = np.insert(np.random.uniform(-1,1, size=1000*2).reshape(1000, 2),0,1,axis=1)

        x1_x2 = (dataset[:, 1]* dataset[:, 2]).reshape(1000,1)
        x1_square = (dataset[:,1]**2).reshape(1000,1)
        x2_square = (dataset[:,2]**2).reshape(1000,1)
        dataset = np.concatenate((dataset, x1_x2,x1_square,x2_square), axis=1)
        flipped_sign= []
        for i in range(len(dataset)):
            sign = nonlinear_target(dataset[i][1],dataset[i][2])
            if i %10 == 0:sign=-1*sign
            flipped_sign.append(sign)

    #     for i in range(len(dataset)):
    #         sign = nonlinear_target(dataset[i][1],dataset[i][2])
    #         sign_list.append(sign)
        count, weight = linear_regression(dataset, flipped_sign)
        weight_total+=weight

    return (weight_total/1000)
# q12()

array([-0.99121303, -0.00000334,  0.0005402 , -0.00572102,  1.55551064,
        1.55659617])

In [228]:

for i in range(1000):
    dataset = np.insert(np.random.uniform(-1,1, size=1000*2).reshape(1000, 2),0,1,axis=1)

    x1_x2 = (dataset[:, 1]* dataset[:, 2]).reshape(1000,1)
    x1_square = (dataset[:,1]**2).reshape(1000,1)
    x2_square = (dataset[:,2]**2).reshape(1000,1)
    dataset = np.concatenate((dataset, x1_x2,x1_square,x2_square), axis=1)
    flipped_sign= []
    for i in range(len(dataset)):
        sign = nonlinear_target(dataset[i][1],dataset[i][2])
        if i %10 == 0:sign=-1*sign
        flipped_sign.append(sign)

#     for i in range(len(dataset)):
#         sign = nonlinear_target(dataset[i][1],dataset[i][2])
#         sign_list.append(sign)
    count, weight = linear_regression(dataset, flipped_sign)
weight


array([-0.91772465,  0.04942707,  0.05448993, -0.0651864 ,  1.46734572,
        1.44471018])

In [240]:


def q12(weight, num_of_iter, num_of_fresh):
    fresh_probs = []
    for i in range( num_of_iter ):
        fresh_points = np.insert(np.random.uniform(-1,1, size=num_of_fresh*2).reshape(num_of_fresh,2),0,1,axis=1)

        x1_x2 = (fresh_points[:, 1]* fresh_points[:, 2]).reshape(1000,1)
        x1_square = (fresh_points[:,1]**2).reshape(1000,1)
        x2_square = (fresh_points[:,2]**2).reshape(1000,1)
        fresh_points = np.concatenate((fresh_points, x1_x2,x1_square,x2_square), axis=1)
        signs_of_fresh = []
        for i in range(len(fresh_points)):
            sign = nonlinear_target(fresh_points[i][1],fresh_points[i][2])
            if i %10 == 0:sign=-1*sign
            signs_of_fresh.append(sign)
        predict = np.sign(np.matmul(weight, fresh_points.T))

        miss_count = 0
        for i in range(len(predict)):
            if int(predict[i]) != int(signs_of_fresh[i]):miss_count+=1
        fresh_probs.append(miss_count / len(fresh_points))

    return np.average(fresh_probs)

# answer is close to 0.1 answer 
q12(weight, 1000, 1000)


0.119383